In [14]:
from tqdm import tqdm
from peewee import SqliteDatabase, fn
from playhouse.shortcuts import model_to_dict

from database import Page

In [16]:
path_data = '/Volumes/ExternalSSD/FakeNewsRecognition/'
peewee_database_merged = SqliteDatabase(path_data + 'news_cleaned_2018_01_29+postgres+nytimes+webhose.db')
peewee_database_webhose = SqliteDatabase(path_data + 'webhose/news_cleaned_webhose.db')
peewee_database_nytimes = SqliteDatabase(path_data + 'nytimes/news_cleaned_nytimes.db')

In [18]:
Page._meta.database = peewee_database_merged
Page.select(fn.Max(Page.id)).scalar()

8335329

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Additional-column" data-toc-modified-id="Additional-column-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Additional column</a></span></li><li><span><a href="#Add-webhose" data-toc-modified-id="Add-webhose-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Add webhose</a></span></li><li><span><a href="#Add-nytimes" data-toc-modified-id="Add-nytimes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Add nytimes</a></span></li></ul></div>

# Additional column

In [5]:
'source'

'source'

In [7]:
def insert_bulk(pages, db):
    Page._meta.database = db
    db.connect(reuse_if_open=True)
    
    with db.atomic():
        Page.insert_many(pages).execute()
        
def fetch_pages(last_id, batch_size, db):
    while True:
        Page._meta.database = db
        db.connect(reuse_if_open=True)
        
        pages = list(Page.select()
                     .where((Page.id > last_id) & (Page.id <= (last_id + batch_size)))
                     .order_by(Page.id.asc()))

        if len(pages) <= 0:
            return

        for page in pages:
            yield page

            last_id = page.id

# Add webhose

In [10]:
batch_size = 50
with tqdm() as progress:
    for page in fetch_pages(0, batch_size, peewee_database_webhose):
        progress.update()

342351it [02:22, 2408.51it/s]


In [11]:
batch_size = 50
pages_to_insert = []
with tqdm() as progress:
    for page in fetch_pages(0, batch_size, peewee_database_webhose):
        _page = model_to_dict(page)
        _page['source'] = 'webhose'
        del _page['id']
        
        pages_to_insert.append(_page)
        progress.update()
        
        if len(pages_to_insert) > batch_size:
            insert_bulk(pages_to_insert, peewee_database_merged)
            pages_to_insert = []   
        
insert_bulk(pages_to_insert, peewee_database_merged)

342351it [05:28, 1042.10it/s]


# Add nytimes

In [19]:
batch_size = 50
with tqdm() as progress:
    for page in fetch_pages(0, batch_size, peewee_database_nytimes):
        progress.update()

21900it [00:11, 1850.80it/s]


KeyboardInterrupt: 

In [20]:
batch_size = 50
pages_to_insert = []
with tqdm() as progress:
    for page in fetch_pages(0, batch_size, peewee_database_nytimes):
        _page = model_to_dict(page)
        _page['source'] = 'nytimes'
        del _page['id']
        
        pages_to_insert.append(_page)
        progress.update()
        
        if len(pages_to_insert) > batch_size:
            insert_bulk(pages_to_insert, peewee_database_merged)
            pages_to_insert = []   
        
insert_bulk(pages_to_insert, peewee_database_merged)

1542967it [27:40, 929.25it/s] 
